<a href="https://colab.research.google.com/github/Abuton/Abuton/blob/gh-pages/Temp_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
# from plotly.online import init_notebook_mode
# init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings("ignore")

In [2]:
global_temp_country = pd.read_csv("/content/drive/MyDrive/Copy of GlobalLandTemperaturesByCountry.csv")

global_temp_country.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
1,1743-12-01,NaN,NaN,Åland
2,1744-01-01,NaN,NaN,Åland
3,1744-02-01,NaN,NaN,Åland
4,1744-03-01,NaN,NaN,Åland


In [3]:
global_temp_country.isna().sum()

dt                                   0
AverageTemperature               32651
AverageTemperatureUncertainty    31912
Country                              0
dtype: int64

In [4]:
global_temp_country.shape

(577462, 4)

In [5]:
global_temp_country.dropna(axis="index", inplace=True, subset=["AverageTemperature", "AverageTemperatureUncertainty"])

In [6]:
global_temp_country.isna().sum()

dt                               0
AverageTemperature               0
AverageTemperatureUncertainty    0
Country                          0
dtype: int64

In [7]:
global_temp_country['Country'].nunique()

242

In [8]:
global_temp_country['Country'].unique()

array(['Åland', 'Afghanistan', 'Africa', 'Albania', 'Algeria',
       'American Samoa', 'Andorra', 'Angola', 'Anguilla',
       'Antigua And Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Asia',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Baker Island', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium',
       'Belize', 'Benin', 'Bhutan', 'Bolivia',
       'Bonaire, Saint Eustatius And Saba', 'Bosnia And Herzegovina',
       'Botswana', 'Brazil', 'British Virgin Islands', 'Bulgaria',
       'Burkina Faso', 'Burma', 'Burundi', "Côte D'Ivoire", 'Cambodia',
       'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands',
       'Central African Republic', 'Chad', 'Chile', 'China',
       'Christmas Island', 'Colombia', 'Comoros',
       'Congo (Democratic Republic Of The)', 'Congo', 'Costa Rica',
       'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic',
       'Denmark (Europe)', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt'

In [9]:
my_map = {
    "Denmark (Europe)": "Denmark",
    "United Kingdom (Europe)": "United Kingdom",
    "Congo (Democratic Republic Of The)": "Congo",
    "France (Europe)": "France",
    "Netherlands (Europe)": "Netherlands"
}

global_temp_country["Country"] = global_temp_country["Country"].replace(my_map)

In [10]:
global_temp_country['Country'].nunique()

237

In [11]:
avg_temp = global_temp_country.groupby("Country")["AverageTemperature"].mean().to_frame().reset_index()
avg_temp

,Country,AverageTemperature
0,Afghanistan,14.045007
1,Africa,24.074203
2,Albania,12.610646
3,Algeria,22.985112
4,American Samoa,26.611965
...,...,...
232,Western Sahara,22.319818
233,Yemen,26.253597
234,Zambia,21.282956
235,Zimbabwe,21.117547


In [12]:
# draw chloropleth map

fig = px.choropleth(avg_temp, locations="Country", locationmode="country names", color="AverageTemperature",
                    hover_data=["Country"])
fig.update_layout(title="Choropleth Map of AVG Temp")

fig.show()

In [13]:
# Is there Global Warming?

global_temp = pd.read_csv("/content/drive/MyDrive/Copy of GlobalTemperatures.csv")

global_temp.head()

,dt,LandAverageTemperature,LandAverageTemperatureUncertainty,LandMaxTemperature,LandMaxTemperatureUncertainty,LandMinTemperature,LandMinTemperatureUncertainty,LandAndOceanAverageTemperature,LandAndOceanAverageTemperatureUncertainty
0,1750-01-01,3.034,3.574,NaN,NaN,NaN,NaN,NaN,NaN
1,1750-02-01,3.083,3.702,NaN,NaN,NaN,NaN,NaN,NaN
2,1750-03-01,5.626,3.076,NaN,NaN,NaN,NaN,NaN,NaN
3,1750-04-01,8.490,2.451,NaN,NaN,NaN,NaN,NaN,NaN
4,1750-05-01,11.573,2.072,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
def fetch_year(date):
  return date.split("-")[0]

In [15]:
global_temp['year'] = global_temp['dt'].apply(fetch_year)

global_temp.head()

,dt,LandAverageTemperature,LandAverageTemperatureUncertainty,LandMaxTemperature,LandMaxTemperatureUncertainty,LandMinTemperature,LandMinTemperatureUncertainty,LandAndOceanAverageTemperature,LandAndOceanAverageTemperatureUncertainty,year
0,1750-01-01,3.034,3.574,NaN,NaN,NaN,NaN,NaN,NaN,1750
1,1750-02-01,3.083,3.702,NaN,NaN,NaN,NaN,NaN,NaN,1750
2,1750-03-01,5.626,3.076,NaN,NaN,NaN,NaN,NaN,NaN,1750
3,1750-04-01,8.490,2.451,NaN,NaN,NaN,NaN,NaN,NaN,1750
4,1750-05-01,11.573,2.072,NaN,NaN,NaN,NaN,NaN,NaN,1750


In [16]:
data1 = global_temp.groupby("year").agg({"LandAverageTemperature": "mean", "LandAverageTemperatureUncertainty": "mean"}).reset_index()

In [17]:
data1["UncertaintyTop"] = data1["LandAverageTemperature"] + data1["LandAverageTemperatureUncertainty"]
data1["UncertaintyBottom"] = data1["LandAverageTemperature"] - data1["LandAverageTemperatureUncertainty"]

data1.head()

,year,LandAverageTemperature,LandAverageTemperatureUncertainty,UncertaintyTop,UncertaintyBottom
0,1750,8.719364,2.637818,11.357182,6.081545
1,1751,7.976143,2.781143,10.757286,5.195000
2,1752,5.779833,2.977000,8.756833,2.802833
3,1753,8.388083,3.176000,11.564083,5.212083
4,1754,8.469333,3.494250,11.963583,4.975083


In [ ]:
fig = px.line(data1,  x="year", y=["LandAverageTemperature", "UncertaintyTop", "UncertaintyBottom"],
              title="Line chart per year Avg Temp")
fig.show()

In [26]:
# analyzing per season

global_temp["dt"] = pd.to_datetime(global_temp["dt"])

global_temp["month"] = global_temp["dt"].dt.month

In [27]:
def get_season(month):
  if month >= 3 and month <= 5:
    return "spring"
  elif month >= 6  and month <= 8:
    return "summer"
  elif month >=9 and month <= 11:
    return "autuum"
  else:
    return "winter"

global_temp["season"] = global_temp["month"].apply(get_season)

global_temp.head(3)

,dt,LandAverageTemperature,LandAverageTemperatureUncertainty,LandMaxTemperature,LandMaxTemperatureUncertainty,LandMinTemperature,LandMinTemperatureUncertainty,LandAndOceanAverageTemperature,LandAndOceanAverageTemperatureUncertainty,year,month,season
0,1750-01-01,3.034,3.574,NaN,NaN,NaN,NaN,NaN,NaN,1750,1,winter
1,1750-02-01,3.083,3.702,NaN,NaN,NaN,NaN,NaN,NaN,1750,2,winter
2,1750-03-01,5.626,3.076,NaN,NaN,NaN,NaN,NaN,NaN,1750,3,spring


In [28]:
years = global_temp['year'].unique()

In [32]:
spring_temp, summer_temp, autumn_temp, winter_temp = [], [], [], []

for year in years:
  current_df = global_temp[global_temp['year'] == year]
  spring_temp.append(current_df[current_df['season'] == "spring"]["LandAverageTemperature"].mean())
  summer_temp.append(current_df[current_df['season'] == "summer"]["LandAverageTemperature"].mean())
  autumn_temp.append(current_df[current_df['season'] == "autuum"]["LandAverageTemperature"].mean())
  winter_temp.append(current_df[current_df['season'] == "winter"]["LandAverageTemperature"].mean())

In [33]:
season = pd.DataFrame()

In [34]:
season['year'] = years
season['summer_temp'] = summer_temp
season['spring'] = spring_temp
season['autumn'] = autumn_temp
season['winter'] = winter_temp

season.head()

,year,summer_temp,spring,autumn,winter
0,1750,14.518333,8.563000,8.890000,2.963000
1,1751,14.116000,6.735000,10.673000,1.729000
2,1752,NaN,7.035500,7.587000,2.717000
3,1753,14.608333,8.627333,9.212333,1.104333
4,1754,14.208333,9.074333,8.957333,1.637333


In [35]:
season.columns

Index(['year', 'summer_temp', 'spring', 'autumn', 'winter'], dtype='object')

In [ ]:
fig = px.line(season, x='year', y=['summer_temp', 'spring', 'autumn', 'winter'], title='Avg Temp')
fig.show()

In [39]:
# Analysis for USA

temp_state = pd.read_csv("/content/drive/MyDrive/Copy of GlobalLandTemperaturesByState.csv")

temp_state.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
0,1855-05-01,25.544,1.171,Acre,Brazil
1,1855-06-01,24.228,1.103,Acre,Brazil
2,1855-07-01,24.371,1.044,Acre,Brazil
3,1855-08-01,25.427,1.073,Acre,Brazil
4,1855-09-01,25.675,1.014,Acre,Brazil


In [40]:
my_filter = temp_state["Country"] == "United States"

usa = temp_state[my_filter].dropna()

usa.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
7458,1743-11-01,10.722,2.898,Alabama,United States
7463,1744-04-01,19.075,2.902,Alabama,United States
7464,1744-05-01,21.197,2.844,Alabama,United States
7465,1744-06-01,25.290,2.879,Alabama,United States
7466,1744-07-01,26.420,2.841,Alabama,United States


In [41]:
usa['State'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District Of Columbia',
       'Florida', 'Georgia (State)', 'Hawaii', 'Idaho', 'Illinois',
       'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine',
       'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [42]:
state_map = {
    'Georgia (State)':'Georgia',
    'District Of Columbia': 'Columbia'
}

usa["State"].replace(state_map, inplace=True)

In [43]:
usa = usa[['AverageTemperature', 'State']]

usa.head()

,AverageTemperature,State
7458,10.722,Alabama
7463,19.075,Alabama
7464,21.197,Alabama
7465,25.290,Alabama
7466,26.420,Alabama


In [46]:
usa_temp = usa.groupby("State").mean().reset_index()

usa_temp.head()

,State,AverageTemperature
0,Alabama,17.066138
1,Alaska,-4.890738
2,Arizona,15.381526
3,Arkansas,15.573963
4,California,14.327677


In [ ]:
!pip install opencage

In [49]:
location_list = usa_temp["State"].tolist()

In [50]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="app")

latitudes, longitudes = [], []
for loc in location_list:
  location = geolocator.geocode(loc)
  if location is None:
    latitudes.append(np.nan)
    longitudes.append(np.nan)
  else:
    latitudes.append(location.latitude)
    longitudes.append(location.longitude)

In [51]:
usa_temp['lat'] = latitudes
usa_temp['lon'] = longitudes

usa_temp.head()

,State,AverageTemperature,lat,lon
0,Alabama,17.066138,33.258882,-86.829534
1,Alaska,-4.890738,64.445961,-149.680909
2,Arizona,15.381526,34.395342,-111.763275
3,Arkansas,15.573963,35.204888,-92.447911
4,California,14.327677,36.701463,-118.755997


In [54]:
import folium
from folium.plugins import HeatMap


In [55]:
basemap = folium.Map()

In [ ]:
HeatMap(usa_temp[['lat', 'lon', 'AverageTemperature']]).add_to(basemap)

basemap

In [58]:
cities = pd.read_csv("/content/drive/MyDrive/Copy of GlobalLandTemperaturesByMajorCity.csv")

cities.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1849-01-01,26.704,1.435,Abidjan,Côte D'Ivoire,5.63N,3.23W
1,1849-02-01,27.434,1.362,Abidjan,Côte D'Ivoire,5.63N,3.23W
2,1849-03-01,28.101,1.612,Abidjan,Côte D'Ivoire,5.63N,3.23W
3,1849-04-01,26.140,1.387,Abidjan,Côte D'Ivoire,5.63N,3.23W
4,1849-05-01,25.427,1.200,Abidjan,Côte D'Ivoire,5.63N,3.23W


In [60]:
my_filter = cities['Country'] == "Nigeria"
nigeria_df = cities[my_filter]

nigeria_df.shape

(5763, 7)

In [62]:
major_cities = nigeria_df['City'].unique()

In [63]:
nigeria_df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
88362,1856-01-01,26.901,1.359,Ibadan,Nigeria,7.23N,4.05E
88363,1856-02-01,27.084,1.717,Ibadan,Nigeria,7.23N,4.05E
88364,1856-03-01,27.754,1.122,Ibadan,Nigeria,7.23N,4.05E
88365,1856-04-01,27.002,1.151,Ibadan,Nigeria,7.23N,4.05E
88366,1856-05-01,25.847,1.308,Ibadan,Nigeria,7.23N,4.05E


In [65]:
nigeria_df["Latitude"] = nigeria_df["Latitude"].str.strip("N")
nigeria_df["Longitude"] = nigeria_df["Longitude"].str.strip("E")

In [66]:
nigeria_df['month'] = pd.to_datetime(nigeria_df['dt']).dt.month

nigeria_df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,month
88362,1856-01-01,26.901,1.359,Ibadan,Nigeria,7.23,4.05,1
88363,1856-02-01,27.084,1.717,Ibadan,Nigeria,7.23,4.05,2
88364,1856-03-01,27.754,1.122,Ibadan,Nigeria,7.23,4.05,3
88365,1856-04-01,27.002,1.151,Ibadan,Nigeria,7.23,4.05,4
88366,1856-05-01,25.847,1.308,Ibadan,Nigeria,7.23,4.05,5


In [ ]:
ng_city_avg = nigeria_df.groupby(['month','City'])["AverageTemperature"].mean().to_frame().reset_index()

ng_city_avg

In [72]:
latitudes, longitudes = [], []
for loc in ng_city_avg['City']:
  location = geolocator.geocode(loc)
  if location is None:
    latitudes.append(np.nan)
    longitudes.append(np.nan)
  else:
    latitudes.append(location.latitude)
    longitudes.append(location.longitude)

In [74]:
ng_city_avg['lat'] = latitudes
ng_city_avg['lon'] = longitudes

ng_city_avg.head()

,month,City,AverageTemperature,lat,lon
0,1,Ibadan,26.861179,7.377746,3.897250
1,1,Kano,21.130172,49.181004,-0.369891
2,1,Lagos,26.206149,6.455057,3.394179
3,2,Ibadan,28.201132,7.377746,3.897250
4,2,Kano,24.052431,49.181004,-0.369891


In [93]:
data1 = ng_city_avg.drop_duplicates(subset=["month", 'City'])

In [94]:
ng2 = data1[["month", "City", "AverageTemperature", "lat", "lon"]]

ng2.head()

,month,City,AverageTemperature,lat,lon
0,1,Ibadan,26.861179,7.377746,3.897250
1,1,Kano,21.130172,49.181004,-0.369891
2,1,Lagos,26.206149,6.455057,3.394179
3,2,Ibadan,28.201132,7.377746,3.897250
4,2,Kano,24.052431,49.181004,-0.369891


In [95]:
ng2.shape

(36, 5)

In [83]:
import plotly.graph_objs as go

In [96]:
data = go.Heatmap(x=ng2['month'],
           y=ng2['City'],
           z=ng2['AverageTemperature'])
layout = go.Layout(title="Major Cities in Nigeria Avg Temp")

fig = go.Figure(data=data, layout=layout)

fig.show()

In [97]:
basemap = folium.Map()

In [98]:
basemap

In [103]:
for id, row in ng2.iterrows():
  folium.Marker(location=[row['lat'], row['lon']], popup='AverageTemperature').add_to(basemap)

basemap